## Part 1

Steps involved :- 
1. Loading Training/Testing Data
2. Formatting into structured JSON format
3. Saving File named "" "updated" + Filename ""

In [6]:
#Loading JSON Library
import json

In [2]:
filename = "f.json"

In [3]:
with open(filename,"r", encoding="utf8") as f:
    data = f.read()

In [4]:
def savetoJSONFormat(data):
    index = data.find("\n")
    data[:index+2]
    
    # no "," after every sentence and overall [] are missing for proper json input
    datareplaced = data.replace("\n",",")
    datareplaced.find("\n")
    
    #above code add "," at last sentence so removing it
    index = len(datareplaced)
    datareplaced = datareplaced[:index-1]
    
    
    with open("updated "+filename,"w+") as f:
        f.write("[" + datareplaced + "]")
        print("File saved :- ", "updated "+filename)
    return 

In [5]:
savetoJSONFormat(data)

File saved :-  updated f.json


# Part 2

Steps invloved :-
1. Loading JSON File of  Traning and Testing Data
2. Formating JSON file to required format for training

{ Loading the New Testing Set (Using On Unseen Data Set)  to avoid overfitting of the training data }

In [7]:
trainfile = "f.json"

In [8]:
with open("updated "+trainfile) as train_data:
    train = json.load(train_data)

In [9]:
TRAIN_DATA = []

for data in train:
    ents = [tuple(entity) for entity in data['labels']]
    TRAIN_DATA.append((data['text'],{'entities':ents}))

print(TRAIN_DATA[0])


('Gujarmal Modi 9 August 1902  22 January 1976 was an Indian industrialist and philanthropist, who co-established the Modi Group of companies and the industrial city of Modinagar in 1933, along with his brother Kedar Nath Modi', {'entities': [(0, 13, 'Person'), (209, 224, 'Person'), (167, 176, 'Location')]})


In [10]:
testfile =  "train1.json"

In [11]:
with open("updated "+testfile) as test_data:
    test = json.load(test_data) 

In [12]:
TEST_DATA = []

for data in test:
    ents = [tuple(entity) for entity in data['labels']]
    TEST_DATA.append((data['text'],{'entities':ents}))
    
print(TEST_DATA[0]) 

('PM Modi and his officials will be holding talks and inaugurate a few defence projects', {'entities': [(0, 7, 'Narendra Modi')]})


# Part  3

Steps involved :- 
1. Loading required library
2. Setting up the parameters for traning
3. Traning and saving the best model

In [13]:
import spacy
import random # random function for to remove bais in Traning Data

# for batch parsing 
from spacy.util import minibatch, compounding

# For evaluateing the model from testing set
from spacy.gold import GoldParse
from spacy.scorer import Scorer


In [5]:
def train_spacy(TRAIN_DATA,TEST_DATA,iterations,droprate = 0.5,modelName = "modelTrained"):
    
    
    modiner = spacy.blank('en')  # create blank Language class
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in modiner.pipe_names:
        ner = modiner.create_pipe('ner')
        modiner.add_pipe(ner, last=True)
     
    # setting up f1score
    f1score = 0.0000

    
    # add labels that will be involved in training 
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
            

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in modiner.pipe_names if pipe != 'ner']
    with modiner.disable_pipes(*other_pipes):  # only train NER
        optimizer = modiner.begin_training()
        
        # --Iterations Starts--
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            #--Shuffling Traning Data--
            random.shuffle(TRAIN_DATA)
            losses = {}
            
                      
                    
            # batch Traning For better Training and Learning of model
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                modiner.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=droprate,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print(losses)
            
            
            # Evaluating the Current Model Score 
            results = evaluate(modiner, TEST_DATA)
            print("Current Score :-",results["ents_f"], "Precision  :-",results["ents_p"], "Recall  :-",results["ents_r"])
#             print("Precision  :-",results["ents_p"])
#             print("Recall  :-",results["ents_r"])

            
            # loading previous best saved model in start of traning 
            if f1score == 0.00:
                try:                    
                    pnlp = spacy.load(modelName)
                    result = evaluate(pnlp, TEST_DATA) # calling evaluate function 
                    f1score = result["ents_f"]
                except:
                    print("Previous Model not found")
                    
            print("Best Sccore :- ",f1score)
            print("------------------------------------")
            # finding out the best score
            if f1score < results["ents_f"]:
                f1score = results["ents_f"]
                # Save our trained Model if the score if grater than best score else no change in previous model
                modiner.to_disk(modelName)
                
    print("-----Best Model is Saved-----")
    return 


In [15]:
def evaluate(ner_model, examples):
    scorer = Scorer()
     
    #loading tags for each input and Evaluating them
    for input_, annotations in examples:
        tags = []
        # loading text
        doc_gold_text = ner_model.make_doc(input_)
        
        #loading all tags for that text
        for ent in annotations.get('entities'):
            tags.append(ent)
            
        # Evaluating the tags    
        gold = GoldParse(doc_gold_text, entities=tags)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
        
        
    return scorer.scores


In [16]:
def loadNERModel(modelName = "modelTrained"):
    pnlp = spacy.load(modelName)
    return pnlp

In [17]:
def f1score(model,TEST_DATA):
    result = evaluate(model, TEST_DATA) # calling evaluate function 
    f1score = result["ents_f"]
    precision = result["ents_p"]
    recall = result["ents_r"]
    print("F1 score of Model is :-",f1score)
    print("Precision of Model is :-",precision)
    print("Recall of Model is :-",recall)

In [ ]:
# traning the model with 50 iterations
train_spacy(TRAIN_DATA,TEST_DATA, 100,droprate = 0.55, modelName = "nm2.0")

In [ ]:
# loading the saved model
pnlp = loadNERModel("nm2.0")

In [ ]:
# calculating the score of the model
f1score(pnlp,TEST_DATA)

In [ ]:
from spacy import displacy

In [ ]:
testcase = pnlp(TEST_DATA[0][0])
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:
for i in range(0,30):
    try:        
        testcase = pnlp(TEST_DATA[i][0])
        displacy.render(testcase, style='ent', jupyter=True)
        print("--")
    except:
        a=None

In [ ]:
trainstring = TRAIN_DATA[0][0]

In [ ]:
for i in range(0,10):
    testcase = pnlp(TRAIN_DATA[i][0])
    displacy.render(testcase, style='ent', jupyter=True)
    print("--")